In [2]:
import pandas as pd
import numpy as np

In [3]:
resort_stats_data = pd.read_csv("moutain_stat.csv")
visitor_data = pd.read_csv("predicted_visitors_2026.csv")
historical_data = pd.read_csv("climate_visitor_snow.csv",index_col = 0)
total_cost = pd.read_csv("total_cost_week_by_week.csv")
print(resort_stats_data.columns)
print(visitor_data.columns)
print(historical_data.columns)
print(total_cost.columns)

Index(['Resort', 'Highest Lifted Point', 'Lowest Lifted Point', 'Skiable Area',
       'Snow Making Area', 'Lifts', 'Terrain Advanced', 'Terrain Intermediate',
       'Terrain Beginner'],
      dtype='object')
Index(['Year', 'Week', 'Resort', 'Predicted_Visitors'], dtype='object')
Index(['Year', 'Week', 'Resort', 'Visitors', 'MaxTemp', 'MinTemp', 'Rainfall',
       'Total Snowfall', 'Snowfall Days', 'Average Base Depth',
       'Max Base Depth', 'Biggest Snowfall'],
      dtype='object')
Index(['Week', 'Mt. Baw Baw', 'Mt. Stirling', 'Mt. Hotham', 'Falls Creek',
       'Mt. Buller', 'Selwyn', 'Thredbo', 'Perisher', 'Charlotte Pass'],
      dtype='object')


In [ ]:
visitor_data.rename(columns = {"Predicted_Visitors":"Forecasted_Visitors"}, inplace = True)
filter_data = historical_data[~historical_data["Resort"].isin(["Mt. Stirling"])]
total_cost = total_cost.loc[:, total_cost.columns != "Mt. Stirling"]
cost_long = total_cost.melt(id_vars=['Week'], var_name='Resort', value_name='Cost')
cost_long.rename(columns = {"Cost":"Price_2025"}, inplace = True)
cost_long.head(5)

,Week,Resort,Price_2025
0,1,Mt. Baw Baw,3381
1,2,Mt. Baw Baw,3381
2,3,Mt. Baw Baw,3381
3,4,Mt. Baw Baw,3381
4,5,Mt. Baw Baw,3556


In [5]:
resort_stats_data.rename(columns={
    'Highest Lifted Point': 'Highest_Lifted_Point',
    'Lowest Lifted Point': 'Lowest_Lifted_Point',
    'Skiable Area': 'Skiable_Area',
    'Snow Making Area': 'Snow_Making_Area',
    'Terrain Advanced': 'Terrain_Advanced',
    'Terrain Intermediate': 'Terrain_Intermediate',
    'Terrain Beginner': 'Terrain_Beginner'
}, inplace=True)
resort_stats_data.head()

,Resort,Highest_Lifted_Point,Lowest_Lifted_Point,Skiable_Area,Snow_Making_Area,Lifts,Terrain_Advanced,Terrain_Intermediate,Terrain_Beginner
0,Perisher,2034,1605,1245,53.4,47,0.18,0.60,0.22
1,Thredbo,2037,1365,480,70.0,14,0.17,0.67,0.16
2,Selwyn,1614,1492,45,36.0,11,0.12,0.48,0.40
3,Charlotte Pass,1955,1760,50,10.0,6,0.20,0.50,0.30
4,Mt. Buller,1805,1375,300,70.0,22,0.35,0.45,0.20


In [ ]:
# ==============================================================================
# PART 1: ASSEMBLE THE RECOMMENDATION DATAFRAME
# ==============================================================================

# --- 1a. Start by merging your core 2026 forecast and price data 
rec_df = pd.merge(visitor_data, cost_long, on=['Resort', 'Week'])
historical_data['Avg_Temp'] = historical_data["MaxTemp"] - historical_data["MinTemp"]
historical_data['Avg_Temp'] = historical_data['Avg_Temp']/2

historical_data['Annual_Avg_Base_Depth'] = historical_data.groupby(['Resort', 'Year'])['Average Base Depth'].transform('mean')

seasonal_multiplier_map = {
    1: 0.20,  # Very early, low base
    2: 0.40,
    3: 0.60,
    4: 0.80,
    5: 1.00,
    6: 1.15,
    7: 1.25,
    8: 1.35,  # Peak season
    9: 1.40,
    10: 1.35,
    11: 1.25,
    12: 1.10,
    13: 0.90,  # Melting season
    14: 0.70,
    15: 0.50
}
historical_data['seasonal_multiplier'] = historical_data['Week'].map(seasonal_multiplier_map)
historical_data['Synthetic_Weekly_Base_Depth'] = historical_data['Annual_Avg_Base_Depth'] * historical_data['seasonal_multiplier']

# --- 1b. Calculate historical averages for ALL condition metrics ---
historical_conditions = historical_data.groupby(['Resort', 'Week']).agg(
    Historical_Avg_Base_Depth=('Synthetic_Weekly_Base_Depth', 'mean'), 
    Historical_Avg_Snowfall_Days=('Snowfall Days', 'mean'),
    Historical_Avg_Rainfall=('Rainfall', 'mean'),
    Historical_Avg_MinTemp=('MinTemp', 'mean')
).reset_index()

# --- 1d. Assemble the final recommendation DataFrame ---
rec_df = pd.merge(visitor_data, cost_long, on=['Resort', 'Week'])
rec_df = pd.merge(rec_df, historical_conditions, on=['Resort', 'Week'])
rec_df = pd.merge(rec_df, resort_stats_data, on='Resort')

print("\n--- Assembled Recommendation DataFrame with SYNTHETIC Snow Depth (Top 5 rows) ---")
rec_df.head()


--- Assembled Recommendation DataFrame with SYNTHETIC Snow Depth (Top 5 rows) ---


,Year,Week,Resort,Forecasted_Visitors,Price_2025,Historical_Avg_Base_Depth,Historical_Avg_Snowfall_Days,Historical_Avg_Rainfall,Historical_Avg_MinTemp,Highest_Lifted_Point,Lowest_Lifted_Point,Skiable_Area,Snow_Making_Area,Lifts,Terrain_Advanced,Terrain_Intermediate,Terrain_Beginner
0,2026.0,1,Charlotte Pass,358,6545,8.933333,18.181818,50.554545,-2.336147,1955,1760,50,10.0,6,0.2,0.5,0.3
1,2026.0,2,Charlotte Pass,477,6545,17.866667,18.181818,32.136364,-3.533766,1955,1760,50,10.0,6,0.2,0.5,0.3
2,2026.0,3,Charlotte Pass,182,6870,26.800000,18.181818,34.536364,-3.893506,1955,1760,50,10.0,6,0.2,0.5,0.3
3,2026.0,4,Charlotte Pass,1342,6870,35.733333,18.181818,23.318182,-4.129221,1955,1760,50,10.0,6,0.2,0.5,0.3
4,2026.0,5,Charlotte Pass,2264,6870,44.666667,18.181818,34.045455,-4.126623,1955,1760,50,10.0,6,0.2,0.5,0.3


In [ ]:

# ==============================================================================
# PART 2: BUILD THE RECOMMENDATION MODEL (WITH FINAL ENHANCED SCORING)
# ==============================================================================

# --- 2a. Normalization functions 
def normalize_positive(series):
    min_val, max_val = series.min(), series.max()
    if max_val == min_val: return pd.Series(1.0, index=series.index)
    return (series - min_val) / (max_val - min_val)

def normalize_negative(series):
    min_val, max_val = series.min(), series.max()
    if max_val == min_val: return pd.Series(1.0, index=series.index)
    return 1 - ((series - min_val) / (max_val - min_val))


# --- 2b. Create Normalized Sub-Scores ---
risky_weeks = [1, 2, 3, 4]
rec_df['is_risky_early_season'] = rec_df['Week'].isin(risky_weeks).astype(int)

norm_base_depth = normalize_positive(rec_df['Historical_Avg_Base_Depth'])
norm_snowfall_days = normalize_positive(rec_df['Historical_Avg_Snowfall_Days'])
norm_rainfall = normalize_negative(rec_df['Historical_Avg_Rainfall'])
norm_temp = normalize_negative(rec_df['Historical_Avg_MinTemp'])

# --- 2d. Create the Final Composite Condition_Score WITH PENALTY ---
early_season_penalty_weight = 0.20
rec_df['Condition_Score'] = (
    (0.50 * norm_base_depth) +
    (0.25 * norm_snowfall_days) +
    (0.15 * norm_temp) +
    (0.10 * norm_rainfall) -
    (early_season_penalty_weight * rec_df['is_risky_early_season']) 
)
rec_df['Condition_Score'] = rec_df['Condition_Score'].clip(lower=0)



rec_df['Crowd_Score'] = normalize_negative(rec_df['Forecasted_Visitors'])
rec_df['Price_Score'] = normalize_negative(rec_df['Price_2025'])
rec_df['Family_Terrain_Score'] = normalize_positive(rec_df['Terrain_Beginner'] + rec_df['Terrain_Intermediate'])
rec_df['Expert_Terrain_Score'] = normalize_positive(rec_df['Terrain_Advanced'] + normalize_positive(rec_df['Highest_Lifted_Point']- rec_df["Lowest_Lifted_Point"]))

rec_df['Intermediate_Terrain_Score'] = normalize_positive(rec_df['Terrain_Intermediate'] + (0.2 * normalize_positive(rec_df['Skiable_Area']))) # Bonus for resort size

In [ ]:
# --- 2c. Define Persona Weights and Apply Scoring ---
persona_weights = {
    'family_fun_seeker': {
        'Family_Terrain_Score': 0.35,
        'Crowd_Score': 0.20,
        'Price_Score': 0.20,
        'Condition_Score': 0.15
    },
    'powder_hound': {
        'Condition_Score': 0.50,
        'Expert_Terrain_Score': 0.40,
        'Crowd_Score': 0.10,
        'Price_Score': 0.10
    },
    'balanced_adventurer': { 
        'Intermediate_Terrain_Score': 0.35,
        'Crowd_Score': 0.30,
        'Condition_Score': 0.20,
        'Price_Score': 0.15
    }
}

for persona, weights in persona_weights.items():
    rec_df[f'{persona}_Ultimate_Score'] = (
        weights.get('Family_Terrain_Score', 0) * rec_df.get('Family_Terrain_Score', 0) +
        weights.get('Intermediate_Terrain_Score', 0) * rec_df.get('Intermediate_Terrain_Score', 0) + 
        weights.get('Expert_Terrain_Score', 0) * rec_df.get('Expert_Terrain_Score', 0) +
        weights.get('Condition_Score', 0) * rec_df['Condition_Score'] +
        weights.get('Crowd_Score', 0) * rec_df['Crowd_Score'] +
        weights.get('Price_Score', 0) * rec_df['Price_Score']
    )

In [ ]:
# ==============================================================================
# PART 3: GENERATE AND DISPLAY RECOMMENDATIONS
# ==============================================================================

holiday_weeks = [5, 6]
# holiday_weeks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
family_options_df = rec_df[rec_df['Week'].isin(holiday_weeks)].copy()

family_recommendation = family_options_df.sort_values(by='family_fun_seeker_Ultimate_Score', ascending=False).reset_index(drop=True)

print("\n\n--- TOP RECOMMENDATIONS for the 'Family Fun-Seeker' (Constrained to Holiday Weeks 5 & 6) ---")
temp_family = family_recommendation[[
    'Resort', 'Week', 'family_fun_seeker_Ultimate_Score',
    'Family_Terrain_Score', 'Crowd_Score', 'Price_Score', 'Condition_Score'
]]
temp_family.head(20)



--- TOP RECOMMENDATIONS for the 'Family Fun-Seeker' (Constrained to Holiday Weeks 5 & 6) ---


,Resort,Week,family_fun_seeker_Ultimate_Score,Family_Terrain_Score,Crowd_Score,Price_Score,Condition_Score
0,Selwyn,6,0.759206,0.965517,0.869482,0.736842,0.666732
1,Selwyn,5,0.753025,0.965517,0.895887,0.736842,0.590320
2,Mt. Baw Baw,6,0.741903,1.000000,0.848922,0.962710,0.197176
3,Mt. Baw Baw,5,0.737070,1.000000,0.842959,0.962710,0.172907
4,Charlotte Pass,6,0.595368,0.689655,0.956120,0.256552,0.743028
5,Charlotte Pass,5,0.587138,0.689655,0.968164,0.256552,0.672104
6,Thredbo,5,0.489913,0.793103,0.355025,0.249734,0.609167
7,Thredbo,6,0.484360,0.793103,0.282379,0.249734,0.669006
8,Perisher,5,0.387213,0.758621,0.142001,0.000000,0.621967
9,Perisher,6,0.380941,0.758621,0.066462,0.000000,0.680877


In [10]:
# --- 3b. Rank and find the winner for the Powder Hound ---
powder_hound_recommendation = rec_df.sort_values(by='powder_hound_Ultimate_Score', ascending=False).reset_index(drop=True)
powder_hound_recommendation[[
    'Resort', 'Week', 'powder_hound_Ultimate_Score',
    'Condition_Score', 'Expert_Terrain_Score', 'Crowd_Score', 'Price_Score']].head(20)

,Resort,Week,powder_hound_Ultimate_Score,Condition_Score,Expert_Terrain_Score,Crowd_Score,Price_Score
0,Mt. Hotham,9,0.866755,0.810818,0.912554,0.609523,0.353718
1,Mt. Hotham,8,0.847046,0.758687,0.912554,0.673092,0.353718
2,Mt. Hotham,10,0.846004,0.765313,0.912554,0.629537,0.353718
3,Mt. Hotham,6,0.840207,0.753103,0.912554,0.632621,0.353718
4,Mt. Hotham,11,0.828074,0.726723,0.912554,0.643189,0.353718
5,Mt. Hotham,13,0.825731,0.626778,0.912554,0.847800,0.625400
6,Mt. Hotham,12,0.822523,0.694583,0.912554,0.748380,0.353718
7,Thredbo,12,0.815344,0.617083,1.000000,0.493129,0.574899
8,Mt. Hotham,5,0.812452,0.695134,0.912554,0.644916,0.353718
9,Thredbo,8,0.811511,0.702985,1.000000,0.350449,0.249734


In [11]:
temp_df=powder_hound_recommendation[[
    'Resort', 'Week', 'powder_hound_Ultimate_Score',
    'Condition_Score', 'Expert_Terrain_Score', 'Crowd_Score', 'Price_Score']]
temp_df[temp_df["Resort"].isin(["Thredbo","Perisher"])]

,Resort,Week,powder_hound_Ultimate_Score,Condition_Score,Expert_Terrain_Score,Crowd_Score,Price_Score
7,Thredbo,12,0.815344,0.617083,1.000000,0.493129,0.574899
9,Thredbo,8,0.811511,0.702985,1.000000,0.350449,0.249734
10,Thredbo,9,0.810648,0.714701,1.000000,0.283235,0.249734
11,Thredbo,13,0.810500,0.552308,1.000000,0.768556,0.574899
13,Thredbo,10,0.807036,0.687919,1.000000,0.381031,0.249734
15,Thredbo,11,0.798210,0.660152,1.000000,0.431612,0.249734
16,Thredbo,7,0.791479,0.667875,1.000000,0.325683,0.249734
17,Thredbo,6,0.787714,0.669006,1.000000,0.282379,0.249734
19,Thredbo,14,0.780498,0.463772,1.000000,0.911222,0.574899
20,Thredbo,5,0.765060,0.609167,1.000000,0.355025,0.249734


In [14]:
# --- 3c. Rank and find the winner for the Balanced Adventurer ---
balanced_adventurer_recommendation = rec_df.sort_values(by='balanced_adventurer_Ultimate_Score', ascending=False).reset_index(drop=True)

print("\n\n--- TOP 5 RECOMMENDATIONS for the 'Balanced Adventurer' ---")
balanced_adventurer_recommendation[[
    'Resort', 'Week', 'balanced_adventurer_Ultimate_Score',
    'Intermediate_Terrain_Score', 'Crowd_Score', 'Condition_Score', 'Price_Score'
]].head(10)



--- TOP 5 RECOMMENDATIONS for the 'Balanced Adventurer' ---


,Resort,Week,balanced_adventurer_Ultimate_Score,Intermediate_Terrain_Score,Crowd_Score,Condition_Score,Price_Score
0,Perisher,14,0.755453,1.000000,0.853615,0.507122,0.319625
1,Thredbo,15,0.751714,0.841238,0.959057,0.416643,0.574899
2,Perisher,15,0.750821,1.000000,0.914734,0.392283,0.319625
3,Thredbo,14,0.746789,0.841238,0.911222,0.463772,0.574899
4,Thredbo,13,0.721697,0.841238,0.768556,0.552308,0.574899
5,Perisher,3,0.715161,1.000000,0.868803,0.282882,0.319625
6,Perisher,13,0.714439,1.000000,0.675483,0.569250,0.319625
7,Perisher,2,0.707102,1.000000,0.869674,0.241283,0.319625
8,Thredbo,3,0.699050,0.841238,0.861541,0.299598,0.574899
9,Thredbo,2,0.691338,0.841238,0.891414,0.216228,0.574899
